<a href="https://colab.research.google.com/github/jarvisx17/EPO/blob/main/EPO_Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/gsong/python-epo-ops-client
!pip install xmltodict
!pip install python-epo-ops-client

In [2]:
import epo_ops
import json
import xmltodict
import re
import os
import sys
import pprint
# from pymongo import MongoClient

In [3]:
module_path = os.path.abspath(os.path.join('python-epo-ops-client')) # or the path to your source code
sys.path.insert(0, module_path)

In [4]:
# ~ Initialize API client for EPO patents ~ #
client = epo_ops.Client(key='wuv2ZOHumPKd7D3FEHVeQeSM819eIWry', secret='iJTsADGX2AAc5pWq')  # Instantiate client

In [5]:
# Date Range of Documents to be retrive
response = client.register_search("pd=20211108-20211213 and classification=g", range_begin=1, range_end=100) 
# print(len(response.text))
data_dict = xmltodict.parse(response.text)
json_data = json.dumps(data_dict)
# print(json_data[:1000])

In [6]:
list_of_patents = []
list_of_epodoc_objects = []
array_of_patents = (data_dict["ops:world-patent-data"]["ops:register-search"]["reg:register-documents"]["reg:register-document"])

for patent in array_of_patents:
#     print(patent['reg:bibliographic-data']["reg:invention-title"])
#     print(patent['reg:bibliographic-data']["reg:publication-reference"]["reg:document-id"])
    patent_number = str(patent['reg:bibliographic-data']["reg:publication-reference"]["reg:document-id"]
         ['reg:country']+patent['reg:bibliographic-data']["reg:publication-reference"]
          ["reg:document-id"]["reg:doc-number"]
         )
    list_of_patents.append(patent_number)
    list_of_epodoc_objects.append(epo_ops.models.Epodoc(patent_number))
#     print(patent['reg:bibliographic-data']["reg:classifications-ipcr"]["reg:classification-ipcr"])

In [7]:
# patents = [epo_ops.models.Epodoc("EP3920660"), epo_ops.models.Epodoc("EP3918902")]
api_response = client.published_data(  # Retrieve bibliography data
  reference_type = 'publication',  # publication, application, priority
  input = list_of_epodoc_objects,  # can be a list of original, docdb, epodoc
  endpoint = 'biblio',  # optional, defaults to biblio in case of published_data
  constituents = []  # optional, list of constituents
)

# this produces a python list with an entry for each patent
response_list=xmltodict.parse(api_response.text)["ops:world-patent-data"]['exchange-documents']['exchange-document']
#optional: override list by cutting down for shorter testing
# response_list=response_list[:10]
# print(len(response_list))
# print(json.dumps(response_list, indent = 4)[:50000])

In [8]:
def getFullText(epodoc):
    # trying with a recently published EPO numbered doc works
    ft_response = client.published_data(  # Retrieve bibliography data
      reference_type = 'publication',  # publication, application, priority
      input = epo_ops.models.Epodoc(epodoc),  # original, docdb, epodoc
      endpoint = 'description',  # fulltext lists what is available (claims, description, both)
      constituents = []  
    )
    return xmltodict.parse(ft_response.text)

def getParagraph(fullText, para):
  if isinstance(para, int):
      para=f"[{para:04d}]"
      # print(para)
  response = {}
  para_list = fullText["description"]["p"]
  for paragraph in para_list:
      if para in paragraph:
          response[para] = paragraph
  return response


def getCitedParagraphs(citation):
    # print("\noriginal citation: ", citation)############################################################################################################
    citedParagraphs = []
    if type(citation) == list:
        if len(citation) == 1:
            return getCitedParagraphs(citation[0])
        # clean up each element of the list
        for part in citation:
            citedParagraphs = citedParagraphs + cleanup(part)
    else:
        citationList=re.split(",|;", citation)
        for part in citationList:
            citedParagraphs = citedParagraphs + cleanup(part)
    # print("citedParagraphs intermediate: ", citedParagraphs)############################################################################################################
    for passage in citedParagraphs:
        if (re.search(r"(?i)fig\S*|columns?|abstract", passage)) or not (re.findall(r"\d", passage)):
            citedParagraphs.remove(passage)
            # print(f"removed %s from paragraph list" % passage)
            continue
        references = re.findall(r"\[\d+\]", passage)
        if len(references) > 2: # some citations are messed up in API version, this splits back into pairs
            citedParagraphs.remove(passage)
            for i in range(0, len(references)-1, 2):
                citedParagraphs.append(re.sub(r"\[|\]", "", references[i]+"-"+references[i+1]))
            if len(references)%2 == 1:
                citedParagraphs.append(re.sub(r"\[|\]", "", references[-1])) # final element
    citedParagraphs = [re.sub(r"\[|\]", "", item) for item in citedParagraphs] # remove remaining square brackets
    
    # print("citedParagraphs: ", citedParagraphs)
    return citedParagraphs

def cleanup(paraString):
    clean = []
    splitString = re.split(",|;", paraString)
    for para in splitString:
        # ignore elements that are not paragraphs (or just numbers which represent paragraphs)
        # (?i) makes it case insensitive
        if not(re.search(r"(?i)fig\S*|columns?|abstract", para)):
            # remove stars, paragraphs, whitespace
            cleanedup = re.sub(r"\*|paragraphs?|\s|;.*$", "", para)
            if (re.search(r"\[\d*\]", cleanedup)): # must contain a number between square brackets
                clean.append(cleanedup)
    return clean

def getClaims(epodoc):
    api_response = client.published_data(  # Retrieve bibliography data
        reference_type='publication',  # publication, application, priority
        input = epo_ops.models.Epodoc(epodoc),  # original, docdb, epodoc
        endpoint='claims',  
        constituents=[]  # optional, list of constituents
    )
    json_response = xmltodict.parse(api_response.text)
    return (json_response['ops:world-patent-data']['ftxt:fulltext-documents']['ftxt:fulltext-document']
        ['claims']['claim']['claim-text'])

def getRangeFromString(rangeString):
    rangeList = []
    if '-' in rangeString:
        itemRange = rangeString.split("-")
        rangeList = (list(range(int(itemRange[0]), int(itemRange[-1]) + 1)))
    else:
        rangeList.append(int(rangeString))
    return rangeList

def checkAndDownloadClaims(response):
    # print(response["bibliographic-data"].keys())
    if not ("claims" in response["bibliographic-data"].keys()):
        response["bibliographic-data"]["claims"] = (getClaims(
                response["bibliographic-data"]["publication-reference"]["document-id"][1]["doc-number"]))
            # print(json.dumps(response["bibliographic-data"], indent=4))
        # print(response["bibliographic-data"].keys())

In [9]:
# this is currently the "main loop" that pulls citations for each
# patent in turn, then gets the full text of patents cited (if available)
# and the claims against which each was cited

count = 0
cit = 0
xcit = 0
ftxcit = 0
dlclaims = 0
text = ''
rel_patent = ''
text_2 = []
claims_list = []
category = []
rel_cat_claims = []
citations_dataset = [] #  this list will hold the patents where all necessary info has been downloaded

for response in response_list:
    count += 1
    # print("\n", "Count: ", count)
    Patent_Nuumber_d = json.dumps(response["bibliographic-data"]["publication-reference"]['document-id'][1]['doc-number'])
    # print("Patent No.: ", json.dumps(response["bibliographic-data"]["publication-reference"]['document-id'][1]['doc-number']))
    #print(getClaims(response["bibliographic-data"]["publication-reference"]["document-id"][1]["doc-number"]))
    if "references-cited" in response["bibliographic-data"]:
        listofcitations = response["bibliographic-data"]["references-cited"]["citation"]
        if isinstance(listofcitations, dict): # catches the case of a single citation, returned as a dict
            listofcitations = [listofcitations]
        # print("Number of citations: ", len(listofcitations))
        cit += len(listofcitations)
        # print(listofcitations) # avoid unless troubleshooting
        # citations by applicant do not have category etc... -> check examiner citation
        download_claims = False
        for citation in listofcitations:
            if (citation["@cited-by"]=="examiner" and "X" in citation["category"] and "patcit" in citation): # consider looking at publications later "nplcit"
                xcit += 1
                # print("Citation details: ", citation["@sequence"], citation["category"], 
                      # citation["rel-claims"], "Cited doc: ", json.dumps(citation["patcit"]["document-id"][0]))
                
                patent_number = citation["patcit"]["document-id"][0]["doc-number"]
                if patent_number[:2] == "EP":
                    rel_patent += citation["patcit"]["document-id"][0]['doc-number'] + ' '
                    checkAndDownloadClaims(response)
                    ftxcit += 1
                    # print("Further processing possible: downloading EP full text")
                    category.append(citation["category"])
                    rel_cat_claims.append(citation["rel-claims"])
                    citation["citation_text"] = getFullText(patent_number)["ops:world-patent-data"]["ftxt:fulltext-documents"]["ftxt:fulltext-document"]
                    citation["rel-passage"]["rel-passage-list"] = getCitedParagraphs(citation["rel-passage"]["passage"])
                    passageSet = set() # using a set to guarantee uniqueness
                    for passages in citation["rel-passage"]["rel-passage-list"]:
                        passageSet.update(set(getRangeFromString(passages)))
                    citation["rel-passage"]["rel-passage-full-set"] = sorted(list(passageSet)) # convert back to list as sets can't go in JSON
                    # print(citation["rel-passage"]["rel-passage-full-set"])
                    citation["rel-passage"]["rel-passage-full-text"]=[]
                    for passage in citation["rel-passage"]["rel-passage-full-set"]:
                        citation["rel-passage"]["rel-passage-full-text"].append(getParagraph(citation["citation_text"], passage))
                    text_2.append(citation["rel-passage"]["rel-passage-full-text"])
                    # print(citation["rel-passage"]["rel-passage-full-text"])
                    # print(json.dumps(citation, indent=4))
                    # turn rel-claims into a list:
                    if ',' in citation["rel-claims"]:
                        claimsList = []
                        for claimItem in citation["rel-claims"].split(","):
                            claimsList.extend(getRangeFromString(claimItem))
                    else:
                        claimsList = [citation["rel-claims"]]
                    citation["rel-claims-list"] = claimsList
                    # print("adding rel-claims-list", claimsList)
                    claims_list.append(claimsList)
                    text += Patent_Nuumber_d + ' '

        if download_claims is True:
            dlclaims += 1
            citations_dataset.append(response)

In [11]:
# patent numbers
patent_numbers = text.replace('"','')
patent_list = patent_numbers.split(' ')
patent_list = patent_list[:-1]

# related Patent
related_Patent = rel_patent.split(' ')
related_Patent = related_Patent[:-1]

In [12]:
claim_list = []
for i in patent_list:
  x = getClaims(i)
  claim_list.append(x)

# EPO Dataframe

In [14]:
import pandas as pd
df = pd.DataFrame({'Patent No.': patent_list, "Cited Patent No." : related_Patent, "Cited Text" : text_2, "Claims" : claim_list, "Rel-Claims No.": claims_list, "Category" : category, "cat_claims" : rel_cat_claims})
# df.to_csv('EPO.csv', index=False)

In [17]:
df['Negative'] = None
df['Positive'] = None

In [45]:
for l in df.index:
  xx = df.Category[l]
  yy = df.cat_claims[l]
  claims = df.Claims[l]

  
  claimmss = []
  rel_claims = []

  # def claimscolumn():
  if type(xx)==list:
    for i in range(len(xx)):
      if xx[i] == 'X' or xx[i] == 'I':
      # if xx[i] == 'A' or xx[i] == 'Y' :
        vv = yy[i]
        vv = vv.split(",")
        for j in vv:
          if ('-' in j) == False:
            claimmss.append(int(j))
            n = int(j)-1
            # print(claims[n])
            rel_claims.append(claims[n])
          if ('-' in j) == True:
            cc = j.split("-")
            for i in range(int(cc[0]), int(cc[1])+1):
              claimmss.append(i)
              n = i-1
              # print(claims[n])
              rel_claims.append(claims[n])
          else:
            # rel_claims.append('')
            pass
  else:
    if xx == 'X' or xx == 'I':
    # if xx == 'A' or  xx == 'Y':
      vv = yy
      vv = vv.split(",")
      for j in vv:
        if ('-' in j) == False:
          claimmss.append(int(j))
          n = int(j)-1
          # print(claims[n])
          rel_claims.append(claims[n])
        if ('-' in j) == True:
          cc = j.split("-")
          for i in range(int(cc[0]), int(cc[1])+1):
            claimmss.append(i)
            n = i-1
            # print(claims[n])
            rel_claims.append(claims[n])
        else:
          # rel_claims.append('')
          pass
  # df['Negative'][l] = rel_claims
  df['Positive'][l] = rel_claims

  # print(claimmss)

In [ ]:
# df

# Negative Dataframe

In [53]:
Negative_df = df[['Cited Text','Negative']]
rows = [0,1,3,4,5,7]
Negative_df.drop(rows, axis=0, inplace=True)
label = ['Negative','Negative']
Negative_df['Label'] = label
Negative_df = Negative_df.explode('Negative')
Negative_df.rename(columns = {'Negative':'rel_Claim'}, inplace = True)
Negative_df

#Positive Dataframe

In [51]:
Positve_df = df[['Cited Text','Positive']]
Positve_df.drop(3, axis=0, inplace=True)
label = ['Positive','Positive','Positive','Positive','Positive','Positive','Positive']
Positve_df['Label'] = label
Positve_df = Positve_df.explode('Positive')
Positve_df.rename(columns = {'Positive':'rel_Claim'}, inplace = True)
Positve_df

# Merge dataFrame 

In [72]:
new_dataframe = pd.concat((Negative_df, Positve_df)).sort_index(axis = 0)
new_dataframe = new_dataframe.sample(frac = 1)
new_dataframe.reset_index(inplace=True)
new_dataframe.to_csv('Final_df_EPO.csv', index=False)

In [73]:
df = pd.read_csv('Final_df_EPO.csv')

In [74]:
def textMerge(x):
  text = ''
  x = eval(x)
  for i in x:
    for j in i:
      text+=i[j]
  return text

In [75]:
df['Cited Text'] = df['Cited Text'].apply(lambda x: textMerge(x))

In [76]:
df.to_csv('Final_df_EPO.csv', index=False)

In [79]:
df

,Cited Text,rel_Claim,Label
0,[0006] Bisher sind Anzeigevarianten für Amp...,14. A computer readable storage medium storing...,Positive
1,[0007] Die der vorliegenden Erfindung zugru...,"4. The method according to claim 3, wherein a ...",Positive
2,[0007] Die der vorliegenden Erfindung zugru...,"13. An electronic device, comprising:\none or ...",Positive
3,"[0018] In one or more embodiments, the firs...",1. An ultra-wide band (UWB) ranging device com...,Positive
4,"[0028] The direct sound calculator 101, the...",9. The sound signal processing device accordin...,Negative
...,...,...,...
64,"[0028] The direct sound calculator 101, the...",11. The sound signal processing device accordi...,Negative
65,[0001] The present invention relates to a m...,"9. Method according to claim 1, wherein in the...",Positive
66,[0007] Die der vorliegenden Erfindung zugru...,15. A computer program comprising instructions...,Positive
67,[0007] Die der vorliegenden Erfindung zugru...,7. An apparatus for outputting signal light in...,Positive
